In [ ]:
# Link to github repo:

import os
import json
import datetime
import csv

import pandas as pd
import gzip

'''
%load_ext lab_black
%matplotlib inline
'''
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

from data_io import File_IO, Database_IO
from data_wrangling import  Data_Wrangling
from data_summarization import Data_Summarization
from setup_config import Setup_Config

In [ ]:
setup = Setup_Config('config.ini') # loads a setup file with variables in .ini format

# if db not usage not wished, unccoment temporarly
database_io = Database_IO(
    host_ip=setup.config.db.host_ip, 
    port=setup.config.db.port, 
    db_user=setup.config.db.db_user, 
    db_pw=setup.config.db.db_pw, 
    db_name=setup.config.db.db_name) # use default postgres db

### 0.  Get overview of all tables and Materialized Views

In [ ]:
# show all tables in schema
sql_query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'public'"
tabls_overview = database_io.query_data(sql_query)
tabls_overview

In [ ]:
# show all materialized views in schema
sql_query = "SELECT * FROM pg_matviews WHERE schemaname = 'public';"
tabls_overview = database_io.query_data(sql_query)
tabls_overview

### 1. Get overview over Variables (Materialized View)
(Access Materialized View for that)

In [ ]:
sql_query = "select * from open_summary_path;"
variable_summary_frame = database_io.query_data(sql_query)
variable_summary_frame

### 2. Get Data from table

In [ ]:
# Here you can apply almost every query

sql_query = "select * from open_uploaded_all limit 10;" # change query for needs, e.g. select with where, aggregate ...
data = database_io.query_data(sql_query)
data

In [ ]:
# Read patient wise

# to make things easier, i have provided a query in the backend in data_io the whole patient.
# The scond call gives data for patient two and so on.

one_patient = database_io.read_next_patient(source='open_uploaded_all', limit_patients=1)
one_patient

#next_patient = database_io.read_next_patient(source='open_uploaded_all', limit_patients=1)
#next_patient

# if there is no more patient we will get an empty frame back